#### Score 30.5601

https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

https://machinelearningmastery.com/how-to-use-the-timeseriesgenerator-for-time-series-forecasting-in-keras/

https://stackabuse.com/time-series-analysis-with-lstm-using-pythons-keras-library/

Tests
https://towardsdatascience.com/time-series-analysis-visualization-forecasting-with-lstm-77a905180eba

https://medium.com/datadriveninvestor/multivariate-time-series-using-rnn-with-keras-7f78f4488679

Exploration keras https://www.kaggle.com/ternaryrealm/lstm-time-series-explorations-with-keras

https://www.dlology.com/blog/how-to-use-keras-timeseriesgenerator-for-time-series-data/

https://www.business-science.io/timeseries-analysis/2018/04/18/keras-lstm-sunspots-time-series-prediction.html

https://stackoverflow.com/questions/42370740/preparing-time-series-data-for-keras-lstm-network-trains-with-extremely-high-l

https://www.altumintelligence.com/articles/a/Time-Series-Prediction-Using-LSTM-Deep-Neural-Networks

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from scipy import stats
from keras.preprocessing.sequence import TimeseriesGenerator

In [40]:
X_train = pd.read_csv('data_dengue/dengue_features_train.csv', index_col=None)
df_y_train = pd.read_csv('data_dengue/dengue_labels_train.csv', index_col=None)
X_test = pd.read_csv('data_dengue/dengue_features_test.csv', index_col=None)

In [41]:
X_train['week_start_date'] = pd.to_datetime(X_train['week_start_date'])
X_train['week_start_date'] = X_train['week_start_date'] - pd.to_datetime('1980-01-01')
X_train['week_start_date'] = pd.to_numeric(X_train['week_start_date'])/100000000000
X_train_new = X_train.fillna(X_train.mean())
X_train_sj=X_train_new[X_train_new.city=='sj']
X_train_iq=X_train_new[X_train_new.city=='iq']
X_train_sj = X_train_sj.drop(['city'], axis=1)
X_train_iq = X_train_iq.drop(['city'], axis=1)

In [42]:
X_test['week_start_date'] = pd.to_datetime(X_test['week_start_date'])
X_test['week_start_date'] = X_test['week_start_date'] - pd.to_datetime('1980-01-01')
X_test['week_start_date'] = pd.to_numeric(X_test['week_start_date'])/100000000000
X_test_new = X_test.fillna(X_test.mean())
X_test_sj=X_test_new[X_test_new.city=='sj']
X_test_iq=X_test_new[X_test_new.city=='iq']
X_test_sj = X_test_sj.drop(['city'], axis=1)
X_test_iq = X_test_iq.drop(['city'], axis=1)

In [43]:
X_train_sj_float = X_train_sj.astype('float64')
X_train_iq_float = X_train_iq.astype('float64')
X_test_sj_float = X_test_sj.astype('float64')
X_test_iq_float = X_test_iq.astype('float64')

In [44]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
X_train_sj_scaled = min_max_scaler.fit_transform(X_train_sj_float)
X_train_iq_scaled = min_max_scaler.fit_transform(X_train_iq_float)
X_test_sj_scaled = min_max_scaler.transform(X_test_sj_float)
X_test_iq_scaled = min_max_scaler.transform(X_test_iq_float)

In [45]:
y_train_sj = np.array(df_y_train[df_y_train.city=='sj']["total_cases"])
y_train_iq = np.array(df_y_train[df_y_train.city=='iq']["total_cases"])

In [51]:
generator_sj = TimeseriesGenerator(X_train_sj_scaled, y_train_sj, length=1, batch_size=1)
generator_iq = TimeseriesGenerator(X_train_iq_scaled, y_train_iq, length=1, batch_size=1)

In [47]:
model_sj = Sequential()
model_sj.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 23)))
model_sj.add(LSTM(50, activation='relu', return_sequences=True))
model_sj.add(LSTM(30, activation='relu'))
model_sj.add(Dense(1))
model_sj.compile(optimizer='adam', loss='mae')

In [50]:
model_sj.fit_generator(generator=generator_sj, steps_per_epoch=10, epochs=30)

Epoch 1/30
10/10 [==============================] - 0s 9ms/step - loss: 39.0246
Epoch 2/30
10/10 [==============================] - 0s 10ms/step - loss: 13.0701
Epoch 3/30
10/10 [==============================] - 0s 9ms/step - loss: 26.3252
Epoch 4/30
10/10 [==============================] - 0s 9ms/step - loss: 51.3995
Epoch 5/30
10/10 [==============================] - 0s 10ms/step - loss: 31.5682
Epoch 6/30
10/10 [==============================] - 0s 10ms/step - loss: 18.8660
Epoch 7/30
10/10 [==============================] - 0s 9ms/step - loss: 55.4592
Epoch 8/30
10/10 [==============================] - 0s 11ms/step - loss: 33.5533
Epoch 9/30
10/10 [==============================] - 0s 12ms/step - loss: 19.7459
Epoch 10/30
10/10 [==============================] - 0s 11ms/step - loss: 14.1763
Epoch 11/30
10/10 [==============================] - 0s 9ms/step - loss: 24.1456
Epoch 12/30
10/10 [==============================] - 0s 12ms/step - loss: 14.0141
Epoch 13/30
10/10 [===========

In [52]:
model_iq = Sequential()
model_iq.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 23)))
model_iq.add(LSTM(50, activation='relu', return_sequences=True))
model_iq.add(LSTM(30, activation='relu'))
model_iq.add(Dense(1))
model_iq.compile(optimizer='adam', loss='mae')

In [53]:
model_iq.fit_generator(generator=generator_iq, steps_per_epoch=10, epochs=30)

Epoch 1/30
10/10 [==============================] - 4s 391ms/step - loss: 7.9990
Epoch 2/30
10/10 [==============================] - 0s 8ms/step - loss: 6.1916
Epoch 3/30
10/10 [==============================] - 0s 9ms/step - loss: 11.0731
Epoch 4/30
10/10 [==============================] - 0s 8ms/step - loss: 4.0611
Epoch 5/30
10/10 [==============================] - 0s 8ms/step - loss: 5.1606
Epoch 6/30
10/10 [==============================] - 0s 8ms/step - loss: 6.3885
Epoch 7/30
10/10 [==============================] - 0s 9ms/step - loss: 6.7299
Epoch 8/30
10/10 [==============================] - 0s 8ms/step - loss: 5.7141
Epoch 9/30
10/10 [==============================] - 0s 8ms/step - loss: 7.6489
Epoch 10/30
10/10 [==============================] - 0s 7ms/step - loss: 7.0792
Epoch 11/30
10/10 [==============================] - 0s 9ms/step - loss: 8.1562
Epoch 12/30
10/10 [==============================] - 0s 13ms/step - loss: 6.9072
Epoch 13/30
10/10 [==========================

In [60]:
generator_predict_sj = TimeseriesGenerator(X_test_sj_scaled, X_test_sj_scaled, length=1, batch_size=1)
generator_predict_iq = TimeseriesGenerator(X_test_iq_scaled, X_test_iq_scaled, length=1, batch_size=1)

In [56]:
predict_sj = model_sj.predict_generator(generator=generator_predict_sj)

In [61]:
predict_iq = model_iq.predict_generator(generator=generator_predict_iq)

In [62]:
result_cases_sj = pd.DataFrame(predict_sj)
result_cases_sj.columns = ['total_cases']
result_cases_sj.head()

,total_cases
0,9.559126
1,7.018054
2,12.366661
3,11.878613
4,11.611992


In [89]:
result_cases_iq = pd.DataFrame(predict_iq)
result_cases_iq.columns = ['total_cases']
result_cases_iq.head()

,total_cases
0,7.993396
1,7.414385
2,8.760790
3,2.794194
4,6.567686


In [86]:
new_row= pd.DataFrame({3})
new_row.columns = ['total_cases']
result_sj = pd.concat([new_row,result_cases_sj]).reset_index().drop(['index'], axis=1)
result_sj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 1 columns):
total_cases    260 non-null float64
dtypes: float64(1)
memory usage: 2.2 KB


In [87]:
result_sj.head()

,total_cases
0,3.000000
1,9.559126
2,7.018054
3,12.366661
4,11.878613


In [90]:
result_iq = pd.concat([new_row,result_cases_iq]).reset_index().drop(['index'], axis=1)
result_iq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 1 columns):
total_cases    156 non-null float64
dtypes: float64(1)
memory usage: 1.3 KB


In [94]:
result_columns_iq.head()

,city,year,weekofyear
260,iq,2010,26
261,iq,2010,27
262,iq,2010,28
263,iq,2010,29
264,iq,2010,30


In [96]:
result_columns_sj = X_test[X_test.city=='sj'][['city','year','weekofyear']]
result_cases_sj = pd.DataFrame(predict_sj)
result_cases_sj.columns = ['total_cases']
result_df_sj = pd.concat([result_columns_sj, result_sj], axis=1)
result_df_sj['total_cases'] = result_df_sj['total_cases'].round().astype(int)
result_df_sj.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,3
1,sj,2008,19,10
2,sj,2008,20,7
3,sj,2008,21,12
4,sj,2008,22,12


In [95]:
result_columns_iq = X_test[X_test.city=='iq'][['city','year','weekofyear']]
result_columns_iq = result_columns_iq.reset_index()
result_df_iq = pd.concat([result_columns_iq, result_iq], axis=1)
result_df_iq['total_cases'] = result_df_iq['total_cases'].round().astype(int)
result_df_iq = result_df_iq.drop(['index'], axis=1)
result_df_iq.head()

,city,year,weekofyear,total_cases
0,iq,2010,26,3
1,iq,2010,27,8
2,iq,2010,28,7
3,iq,2010,29,9
4,iq,2010,30,3


In [97]:
result = pd.concat([result_df_sj,result_df_iq], axis=0)
result.head()
result.to_csv('lstm_2_city_1_slice.csv', index=False)